# Custom Representations – Advanced Exercises

In these exercises you'll practice creating and parsing custom string representations
for `date`, `time`, and `datetime` objects using:

- `strftime`
- `strptime`
- `isoformat` / `fromisoformat`
- Some light pre/post processing around these core tools

Try to solve each exercise **before** looking at the solution.


In [1]:
from __future__ import annotations

from dataclasses import dataclass
from datetime import datetime, date, time, timezone, timedelta
import re


## Exercise 1 – Human-Readable Log Entries

Write a function `format_log_entry` that takes:

- `dt: datetime`
- `level: str` (e.g. `"INFO"`, `"WARNING"`, `"ERROR"`)
- `message: str`

and returns a string formatted exactly like this:

```
[2020-05-15 22:30:45] (INFO) User 'bob' logged in
```

Formatting rules:

- Date format: `YYYY-MM-DD`
- Time format: `HH:MM:SS` (24-hour)
- The level must be upper-cased (even if the input isn't)
- There must be a single space after the closing `]` and after the closing `)`

Use `strftime` for the date/time portion.


In [2]:
def format_log_entry(dt: datetime, level: str, message: str) -> str:
    """YOUR CODE HERE

    Hint:
    - Use dt.strftime(...)
    - Remember to normalize the log level to uppercase
    """
    raise NotImplementedError()


### Exercise 1 – Solution


In [3]:
def format_log_entry(dt: datetime, level: str, message: str) -> str:
    """Format a log entry as:

    [YYYY-MM-DD HH:MM:SS] (LEVEL) message
    """
    timestamp = dt.strftime("%Y-%m-%d %H:%M:%S")
    normalized_level = level.upper()
    return f"[{timestamp}] ({normalized_level}) {message}"


# quick sanity checks
example_dt = datetime(2020, 5, 15, 22, 30, 45)
print(format_log_entry(example_dt, "info", "User 'bob' logged in"))
print(format_log_entry(example_dt, "ERROR", "Something bad happened"))


[2020-05-15 22:30:45] (INFO) User 'bob' logged in
[2020-05-15 22:30:45] (ERROR) Something bad happened


## Exercise 2 – Parsing Multiple Date Formats

You receive date strings from different systems, each using one of
these formats:

1. `'2020-05-15'` → `YYYY-MM-DD`
2. `'15/05/2020'` → `DD/MM/YYYY`
3. `'May 15, 2020'` → `Month DD, YYYY`

Write a function `parse_flexible_date(s: str) -> date` that:

- Tries each of the formats **in that order**
- Returns a `date` object on success
- Raises a `ValueError` with a helpful message if none of the formats match

Use `datetime.strptime` under the hood.


In [4]:
def parse_flexible_date(s: str) -> date:
    """YOUR CODE HERE

    Hint:
    - Define a list of format strings
    - Loop and try datetime.strptime(...)
    - On success, return .date()
    - On failure, raise ValueError(...) at the end
    """
    raise NotImplementedError()


### Exercise 2 – Solution


In [5]:
def parse_flexible_date(s: str) -> date:
    """Parse a date string using multiple allowed formats.

    Supported formats (in order):
    - YYYY-MM-DD
    - DD/MM/YYYY
    - Month DD, YYYY
    """
    formats = [
        "%Y-%m-%d",       # 2020-05-15
        "%d/%m/%Y",       # 15/05/2020
        "%B %d, %Y",      # May 15, 2020
    ]

    last_error: ValueError | None = None

    for fmt in formats:
        try:
            return datetime.strptime(s, fmt).date()
        except ValueError as ex:
            last_error = ex

    raise ValueError(
        f"Could not parse date {s!r} using supported formats: {', '.join(formats)}"
    ) from last_error


# quick sanity checks
for s in ["2020-05-15", "15/05/2020", "May 15, 2020"]:
    print(s, "→", parse_flexible_date(s))


2020-05-15 → 2020-05-15
15/05/2020 → 2020-05-15
May 15, 2020 → 2020-05-15


## Exercise 3 – Normalizing ISO 8601 Time Zone Variants

Python's `datetime.fromisoformat` can parse strings like:

```python
"2020-05-15T22:30:00-05:00"
```

but **not** these equally valid ISO 8601 variants:

- `'2020-05-15T22:30:00-0500'`
- `'2020-05-15T22:30:00-05'`

Write a function `parse_relaxed_iso8601(ts: str) -> datetime` that:

1. Accepts all three variants above
2. Internally **normalizes** the input string into a form accepted
   by `datetime.fromisoformat`
3. Raises `ValueError` if the input cannot be parsed as a datetime

You may:

- Assume the date and time part (`YYYY-MM-DDTHH:MM:SS`) is valid
- Focus only on normalizing the time zone part
- Use regular expressions if you like (recommended)


In [6]:
def parse_relaxed_iso8601(ts: str) -> datetime:
    """YOUR CODE HERE

    Recommended approach:
    - Try datetime.fromisoformat(ts) first
    - If that fails:
        * Normalize 'Z' to '+00:00'
        * Use regex to capture and normalize offset
    """
    raise NotImplementedError()


### Exercise 3 – Solution


In [7]:
_RELAXED_ISO_OFFSET_RE = re.compile(
    r"^(?P<base>\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2})"
    r"(?P<sign>[+-])"
    r"(?P<hour>\d{2})"
    r"(?P<minute>\d{2})?$"
)


def _normalize_iso_offset(ts: str) -> str:
    try:
        datetime.fromisoformat(ts)
        return ts
    except ValueError:
        pass

    if ts.endswith("Z"):
        return ts[:-1] + "+00:00"

    m = _RELAXED_ISO_OFFSET_RE.match(ts)
    if not m:
        raise ValueError(f"Not a supported ISO 8601 timestamp: {ts!r}")

    base = m.group("base")
    sign = m.group("sign")
    hour = m.group("hour")
    minute = m.group("minute") or "00"

    return f"{base}{sign}{hour}:{minute}"


def parse_relaxed_iso8601(ts: str) -> datetime:
    normalized = _normalize_iso_offset(ts)
    return datetime.fromisoformat(normalized)


examples = [
    "2020-05-15T22:30:00-05:00",
    "2020-05-15T22:30:00-0500",
    "2020-05-15T22:30:00-05",
    "2020-05-15T22:30:00+02",
    "2020-05-15T22:30:00Z",
]

for e in examples:
    try:
        parsed = parse_relaxed_iso8601(e)
        print(e, "→", parsed, parsed.tzinfo)
    except ValueError as ex:
        print(e, "→ ERROR:", ex)


2020-05-15T22:30:00-05:00 → 2020-05-15 22:30:00-05:00 UTC-05:00
2020-05-15T22:30:00-0500 → 2020-05-15 22:30:00-05:00 UTC-05:00
2020-05-15T22:30:00-05 → 2020-05-15 22:30:00-05:00 UTC-05:00
2020-05-15T22:30:00+02 → 2020-05-15 22:30:00+02:00 UTC+02:00
2020-05-15T22:30:00Z → 2020-05-15 22:30:00+00:00 UTC


## Exercise 4 – Custom Round-Trip Representation

Format:

```
YYYYMMDD_HHMMSS±HHMM
```

Implement:

1. `to_compact_timestamp(dt: datetime) -> str`
2. `from_compact_timestamp(s: str) -> datetime`

Strict parsing. Must round-trip exactly.


In [8]:
def to_compact_timestamp(dt: datetime) -> str:
    raise NotImplementedError()

def from_compact_timestamp(s: str) -> datetime:
    raise NotImplementedError()


### Exercise 4 – Solution


In [9]:
_COMPACT_TS_RE = re.compile(
    r"^(?P<date>\d{8})_"
    r"(?P<time>\d{6})"
    r"(?P<sign>[+-])"
    r"(?P<hour>\d{2})"
    r"(?P<minute>\d{2})$"
)

def to_compact_timestamp(dt: datetime) -> str:
    if dt.tzinfo is None or dt.utcoffset() is None:
        raise ValueError("dt must be timezone-aware")

    base = dt.strftime("%Y%m%d_%H%M%S")

    offset = dt.utcoffset()
    total_minutes = int(offset.total_seconds() // 60)
    sign = "+" if total_minutes >= 0 else "-"
    total_minutes = abs(total_minutes)
    hours, minutes = divmod(total_minutes, 60)

    return f"{base}{sign}{hours:02d}{minutes:02d}"


def from_compact_timestamp(s: str) -> datetime:
    m = _COMPACT_TS_RE.match(s)
    if not m:
        raise ValueError(f"Invalid compact timestamp: {s!r}")

    date_part = m.group("date")
    time_part = m.group("time")
    sign = m.group("sign")
    hour = int(m.group("hour"))
    minute = int(m.group("minute"))

    year = int(date_part[0:4])
    month = int(date_part[4:6])
    day = int(date_part[6:8])
    hh = int(time_part[0:2])
    mm = int(time_part[2:4])
    ss = int(time_part[4:6])

    total_minutes = hour * 60 + minute
    if sign == "-":
        total_minutes = -total_minutes

    tz = timezone(timedelta(minutes=total_minutes))
    return datetime(year, month, day, hh, mm, ss, tzinfo=tz)


aware_dt = datetime(2020, 5, 15, 22, 30, 45, tzinfo=timezone(timedelta(hours=-5)))
ts = to_compact_timestamp(aware_dt)
rt = from_compact_timestamp(ts)

print("Original:", aware_dt)
print("Compact:", ts)
print("Round trip:", rt)
print("Equal:", aware_dt == rt)


Original: 2020-05-15 22:30:45-05:00
Compact: 20200515_223045-0500
Round trip: 2020-05-15 22:30:45-05:00
Equal: True
